In [1]:
import os
import time
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import category_encoders as ce

from Dataset_Construction import Balance_Ratio
from Sampling import label_divide

os.chdir('C:/Users/user/Desktop/Darui_R08621110')  
os.getcwd()

'C:\\Users\\user\\Desktop\\Darui_R08621110'

In [2]:
def supervise_encoder(train, test, encoder, column_list, label = 'GB') :
    
    encoder_list = ['target', 'catboost', 'leave one out']
    
    if encoder == encoder_list[0] :
        enc = ce.target_encoder.TargetEncoder(cols = column_list)
    elif encoder == encoder_list[1] :
        enc = ce.cat_boost.CatBoostEncoder(cols = column_list)
    elif encoder == encoder_list[2] :
        enc = ce.leave_one_out.LeaveOneOutEncoder(cols = column_list)
        
    train_x, train_y, test_x, test_y = label_divide(train, test, label)
    train_enc = enc.fit_transform(train_x, train_y)
    test_enc = enc.transform(test_x, test_y)
    train_done = pd.concat([train_enc, train_y], axis = 1)
    test_done = pd.concat([test_enc, test_y], axis = 1)
    
    return train_done, test_done


def reverse_onehot(undone_data, keep_data) :
    
    def get_onehot(row):
        vector = []
        for col in undone_data.columns:
            if row[col] == 1:
                vector.append(col)
        return vector

    reverse_matrix = undone_data.apply(get_onehot, axis = 1)
    feature_table = pd.DataFrame(reverse_matrix.to_list())
    feature_table = pd.concat([keep_data[['id']], feature_table], axis = 1)
    reverse_table = pd.melt(feature_table, id_vars = ['id'])
    reverse_table = reverse_table.loc[reverse_table.value.values != None].reset_index(drop = True).drop(columns = 'variable')
    reverse_all = reverse_table.merge(keep_data, on = 'id', how = 'left')
    
    return reverse_all

## Data Processing

### Event data

### Runhist data

In [9]:
##### runhist data #####
train_runhist = pd.read_csv('train_runhist.csv').iloc[:, 1:]
test_runhist = pd.read_csv('test_runhist.csv').iloc[:, 1:]
all_runhist = pd.concat([train_runhist, test_runhist], axis = 0)
print('\n', 'train_runhist:', train_runhist.shape, '\n', 'test_runhist:', test_runhist.shape)


 train_runhist: (80518, 177) 
 test_runhist: (56358, 177)


#### reverse the one-hot features

In [10]:
##### runhist data with features need to be encode #####
allcols = train_runhist.columns.to_list()
subcols = allcols[:1] + allcols[122:-1]
keepcols = allcols[:122] + allcols[-1:]

train_undone = train_runhist[subcols]
test_undone = test_runhist[subcols]
train_keep = train_runhist[keepcols]
test_keep = test_runhist[keepcols]

In [11]:
train_reverse = reverse_onehot(train_undone, train_keep)
test_reverse = reverse_onehot(test_undone, test_keep)

#### encoding

In [12]:
encode_train, encode_test = supervise_encoder(train_reverse, test_reverse, encoder = 'catboost', column_list = ['value'])

encode_train = encode_train.rename(columns = {'value': 'op_catboost'})
encode_test = encode_test.rename(columns = {'value': 'op_catboost'})

mean_train = encode_train.groupby('id').mean().reset_index()
mean_test = encode_test.groupby('id').mean().reset_index()

mean_train.to_csv('train_encode.csv')
mean_test.to_csv('test_encode.csv')

C:\Users\user\anaconda3\envs\aging\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
